In [2]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import mlflow

# Step 1: Load the dataset
print("Loading dataset...")
dataset = load_dataset("imdb")

# Step 2: Tokenize the data
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Step 3: Load pre-trained BERT model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Step 4: Set training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,  # Keep it 1 epoch for quick testing
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
)

# Step 5: Set up Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"].shuffle(seed=42).select(range(1000)),  # Subset for speed
    eval_dataset=tokenized_datasets["test"].shuffle(seed=42).select(range(100)),
)

# Step 6: Log experiment with MLflow
mlflow.start_run()
mlflow.log_params({"learning_rate": 2e-5, "batch_size": 8, "epochs": 1})
trainer.train()
mlflow.log_artifact("./results")
mlflow.end_run()

# Step 7: Save the model
model.save_pretrained("sentiment_model")
tokenizer.save_pretrained("sentiment_model")
print("Model training complete and saved.")


Loading dataset...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:


Abort: 

In [10]:
from transformers import pipeline

# Load pre-trained Hugging Face sentiment analysis pipeline
classifier = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

# Save the model locally
model_path = "sentiment_model"
classifier.model.save_pretrained(model_path)
classifier.tokenizer.save_pretrained(model_path)

print(f"Model saved to {model_path}")



config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Model saved to sentiment_model


In [15]:
from fastapi import FastAPI
from pydantic import BaseModel
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
import nest_asyncio
from pyngrok import ngrok
import uvicorn

# Step 1: Load the model and tokenizer
model_path = "sentiment_model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# Step 2: Initialize FastAPI and the pipeline
app = FastAPI(title="Sentiment Analysis API")
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# Define input schema
class InputText(BaseModel):
    text: str

# Step 3: Create a POST endpoint
@app.post("/predict/")
async def predict_sentiment(input_text: InputText):
    result = classifier(input_text.text)
    return {"label": result[0]['label'], "score": result[0]['score']}

# Step 4: Run FastAPI using Uvicorn and expose it via ngrok
nest_asyncio.apply()
public_url = ngrok.connect(8000).public_url
print(f"Public URL: {public_url}")

uvicorn.run(app, host="0.0.0.0", port=8000)


Public URL: https://1662-35-185-39-132.ngrok-free.app


INFO:     Started server process [520]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     37.99.66.91:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     37.99.66.91:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     37.99.66.91:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     37.99.66.91:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     37.99.66.91:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     20.25.0.130:0 - "POST /predict/ HTTP/1.1" 200 OK
INFO:     20.25.0.130:0 - "POST /predict/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [520]


In [14]:
!ngrok config add-authtoken 2qOl84FSWXQK7cx0ukixPkoekBk_2Lo9RAvcPLved8UZgDyas

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
